In [65]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import random
import pickle

In [133]:
def run_model():
    train_ds = keras.preprocessing.image_dataset_from_directory(
        "Cotton_Leaf_Dataset/cotton/train/",
        validation_split=0.2,
        subset="training",
        seed=1337,
        image_size=(256, 256),
        batch_size=32,
    )
    
    val_ds = keras.preprocessing.image_dataset_from_directory(
        "Cotton_Leaf_Dataset/cotton/val/",
        validation_split=0.2,
        subset="validation",
        seed=1337,
        image_size=(256, 256),
        batch_size=32,
    )
    
    num_classes = 4
    train_ds = train_ds.map(lambda x, y: (x, tf.one_hot(y, num_classes)))

    # Assuming `val_ds` is a `BatchDataset` containing image batches and labels
    val_ds = val_ds.map(lambda x, y: (x, tf.one_hot(y, num_classes)))
    # Data augmentation
    data_augmentation = keras.Sequential(
        [
            layers.experimental.preprocessing.RandomFlip("horizontal"),
            layers.experimental.preprocessing.RandomRotation(0.1),
            layers.experimental.preprocessing.RandomZoom(0.1),
        ]
    )

    # Define the model
    model = keras.Sequential([
        data_augmentation,
        layers.experimental.preprocessing.Rescaling(1./255),
        layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(256,256,3)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    epochs = 50
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs
    )

    # Evaluate the model
    test_ds = keras.preprocessing.image_dataset_from_directory(
        "Cotton_Leaf_Dataset/cotton/test/",
        seed=1337,
        image_size=(256, 256),
        batch_size=32,
    )
    
    test_ds = test_ds.map(lambda x, y: (x, tf.one_hot(y, num_classes)))
    test_loss, test_acc = model.evaluate(test_ds, verbose=2)
    print("Test accuracy:", test_acc)
    model.save("cnn_fyp2.h5")

In [134]:
def main():
    run_model()

In [135]:
if __name__ == "__main__":
    main()

Found 1630 files belonging to 4 classes.
Using 1304 files for training.
Found 47 files belonging to 4 classes.
Using 9 files for validation.
Epoch 1/50
41/41 [==============================] - 74s 2s/step - loss: 1.7305 - accuracy: 0.4256 - val_loss: 0.8778 - val_accuracy: 0.4444
Epoch 2/50
41/41 [==============================] - 67s 2s/step - loss: 0.8481 - accuracy: 0.6664 - val_loss: 0.6699 - val_accuracy: 0.6667
Epoch 3/50
41/41 [==============================] - 68s 2s/step - loss: 0.7233 - accuracy: 0.7025 - val_loss: 0.5715 - val_accuracy: 0.6667
Epoch 4/50
41/41 [==============================] - 68s 2s/step - loss: 0.7178 - accuracy: 0.7209 - val_loss: 0.4798 - val_accuracy: 0.7778
Epoch 5/50
41/41 [==============================] - 70s 2s/step - loss: 0.4944 - accuracy: 0.8067 - val_loss: 0.4158 - val_accuracy: 0.8889
Epoch 6/50
41/41 [==============================] - 63s 2s/step - loss: 0.4481 - accuracy: 0.8359 - val_loss: 0.5162 - val_accuracy: 0.7778
Epoch 7/50
41/41 [=